In [4]:
import numpy as np
import pandas as pd
import ast

movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

#movies.head(1)

#genres
#id
#keywords
#title
#overview
#cast
#crew
movies=movies.merge(credits, on='title')
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

movies.dropna(inplace=True)

#def convert(text)
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 

movies['genres']=movies['genres'].apply(convert)

In [5]:
movies['keywords']=movies['keywords'].apply(convert)

In [6]:
#movies['cast'][0]

def convert3(text):
    L = []
    counter=0;
    for i in ast.literal_eval(text):
        if counter !=3:
            L.append(i['name'])
            counter +=1;
        else :
            break;
    return L 

movies['cast']= movies['cast'].apply(convert3)

In [7]:
import ast

def fetch_director(text):
    L = []
    try:
        data = ast.literal_eval(text)  # safely convert string → list of dicts
        for i in data:
            if i.get('job') == 'Director':
                L.append(i.get('name'))
                break
    except:
        pass
    return L


In [8]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [9]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1


In [10]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)


In [11]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [12]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [13]:
newSet = movies.drop(columns=['overview','genres','keywords','cast','crew'])
newSet['tags'] = newSet['tags'].apply(lambda x: " ".join(x))
newSet['tags']=newSet['tags'].apply(lambda x:x.lower())

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [15]:
vector =cv.fit_transform(newSet['tags']).toarray()

#vector.shape


In [16]:
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
similarity =cosine_similarity(vector)

In [18]:
newSet[newSet['title'] == 'The Lego Movie'].index[0]

np.int64(744)

In [19]:
def recommend(movie):
    index = newSet[newSet['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(newSet.iloc[i[0]].title)
        
    
recommend('Avatar')

Titan A.E.
Small Soldiers
Ender's Game
Independence Day
Aliens vs Predator: Requiem


In [20]:
import pickle

In [21]:
pickle.dump(newSet,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))


In [22]:
newSet.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


In [23]:
newSet.columns


Index(['movie_id', 'title', 'tags'], dtype='object')